## OE4DOS 2021/2022 -prvi domaci zadatak 
### Student: Veljko Ilic 2017/0611

## 1.1 <span style="color:yellow">Implementacija <em>dress_queen_still</em> funkcije koja menja dezen kaputa.</span>

In [ ]:
from pylab import *

import skimage
from skimage import color
from skimage import exposure
from skimage import filters
from skimage import io
import imageio
from skimage import img_as_float

from scipy import ndimage

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import numpy as np

## Izvestaj 1.1 -   Deo koji se odnosi na implementaciju funcije koja menja haljinu kraljici
### U samoj funkciji sam prvo sliku ugradjenom funkcijom prvo preveo u HSV kolor sistem da bih sto lakse izdvojio region od interesa. Nacin na koji sam isprva odredjivao vrednsoit praga na ravnima H i S je pomocu vidzeta medjutim to sam izbacio iz razloga sto mi je pogodniji nacin da ogranicim opseg na histogramu nego da ga samo podelim levo i desno. Imao sam problem sto mi je cvece sa abazura kod leve ruke detektovalo kao haljinu, medjutim kada sam opseg Hjua pomerio malo udesno o udnosu na prag za saturaciju i ako tre dve maske pomnozim cvece sada ispada iz price i ostaje mi samo obris haljine. Maske S, H ravni i finalna maska ce se pojavativi kada pokrenete celiju ispod. 

### Masku primenjujem na H i S ravni nove pozadine koju hocu da postavim kao novu haljinu. Vrednosti V ravni sam smao prepisao sa stare slike da bi mi ostala infomracija o senkama. Medjutim kod nekih pozadina koje su svetlije stara V ravan ume da zamuti boju haljine koju zelimo pa sam ostavio zakomentarisano kako bih menja za druge pozadine malo svetlije, naime kombinovao sam vrednsoti V ravni stare i nove slike.

### Na kraju sam da bih malo ispeglao ivice primenio Gausov filtar 5*5 on mi daje fin rezultat, pa ostale filtre nisam ni pokusavao da stavim. Dakle na ramenima mi nestane naglie prelaz, a i zelena koja ostane abog nesavrsenosti maske postaje manje vise neprimetna. Mora se reci da joj je lice osataje malo zamuceno kao rez filtriranje.

In [ ]:
def dress_queen_still(img_in,img_new):
    
    #Prelazak iz RGB kolor sistema u HSV radi lakse detekcije oblika
    imgHSV = color.rgb2hsv(img_in);
    
    #Odredjivanje maske za H i S ravan sa parametriam koje sam rucno odredio
    imgHtreshold = (imgHSV[:,:,0] > 0.38) & (imgHSV[:,:,0] < 0.75);    
    imgStreshold = imgHSV[:,:,1] > 0.35;
    
    #Kada dobijem te dve maske za H i S ravan ja ih pomnozim i dobijem finalnu masku
    mask = imgHtreshold*imgStreshold;
    
    figure;
    fig, ax = plt.subplots(ncols=3, figsize=(24,8), dpi=80);
    ax[0].imshow(imgStreshold, cmap='gray',vmin=0, vmax=1); ax[0].set_title('Maska S ravni'); ax[0].axis('off');
    ax[1].imshow(imgHtreshold, cmap='gray',vmin=0, vmax=1); ax[1].set_title('Maska H ravni'); ax[1].axis('off');
    ax[2].imshow(mask, cmap='gray',vmin=0, vmax=1); ax[2].set_title('Finalna maska'); ax[2].axis('off');
    #Pomocna promenljiva u koju ce biti smestena nova slika
    image = np.zeros(img_in.shape);
    
    #Isecena slika koju zelim da stavim umesto kaputa tako da bude istih dimenzija kao i queen)dress.jpg
    imageDownscale = img_new[np.arange(0,1080),:][:,np.arange(0,816)]
    
    #Prevodjenje isecene slike u HSV kolor sistem
    imgHSVD = color.rgb2hsv(imageDownscale)
    
    #U novu sliku prepisem sve vrednosti stare za pocetak
    image[:,:,2] = imgHSV[:,:,2];
    image[:,:,1] = imgHSV[:,:,1];
    image[:,:,0] = imgHSV[:,:,0];
    
    #Primenjivanje maske na V ravan
    #image[:,:,2][mask] = imgHSVD[:,:,2][mask]*0.2 + imgHSV[:,:,2][mask]*0.8
    
    #Primenjivanje maske na H i S ravan
    image[:,:,1][mask] = imgHSVD[:,:,1][mask]
    image[:,:,0][mask] = imgHSVD[:,:,0][mask]

    image = color.hsv2rgb(image)
    
    #Niskofrekventno filtriranje za bi se slika bolje blendovala
    img_gauss = filters.gaussian(image, sigma=1, truncate=5)
    
    return img_gauss

new_queen_dress = dress_queen_still(imread('queen_dress.jpg'),imread('pattern.png'))

fig, ax = plt.subplots(figsize=(10,4), dpi=150);
imsave('new_queen.jpg',new_queen_dress);
ax.set_title('RGB', fontsize=14);
ax.imshow(new_queen_dress);

## Izvestaj 1.2 -   Deo koji se odnosi na implementaciju funkcije koja se bavi videom na kom je kraljica

### O ovom delu nema sta puno da kazem nisam ga zavrsio, Intel i3 sa 4 GB rama sa dual boot Ubuntuom samo se zamrzne kada pokrenem skriptu sa vezbi pa nisam ni pokusavao. Svakako ja cu svoju ideju zapisati ovde. Prvo sam procitao da je 30 frejmova po sekundi i kako je to statisticki podatak ja bih ga iskoristio tako sto od frejma 1051 do frejma 1776 ne bih primenjivao masku uopste jer kraljica nije u tim kadrvoima ili od 35 sekunde do 59 sekunde, jedna sekunda gore dole. Manje vise isto kao prethodna tacka stim da bas filtriranje frejmova nakon primene maske ne znam da li bih radio jer to bi onda jos duze trajao. Sto si tice nacina izdvajanje maske iz liste frames[] bih izdvojio par frejmova preveo ih u HSV kolor sistem malo se igra sa parametrima i to bi mi bila maska. Pozadanu sam pripremio neku zutu monotonu.

In [ ]:
def dress_queen_video(new_coat_color):
    
    #Isecena slika koju zelim da stavim umesto kaputa tako da bude istih dimenzija kao i queen)dress.jpg
    coat_downscale = new_coat_color[np.arange(0,640),:][:,np.arange(0,368)]
    
    return new_queens_coat

new_queens_coat = dress_queen_video(imread('new_coat_color.jpg'));

## 2 <span style="color:yellow">Izostravanje <em>miner.jpg</em> fotografije.</span>

## Izvestaj 2.1 -   Deo koji se odnosi na izostravanje slike rudara
### Za potreve izostravanje ove slike sam probao oba metoda i Gradijent i Maskiranje neostrih delova slike pa cu ovde prokomentarisati rezultate oba nacina.
### Za gradijent i prvu matricu koju sam probao(cetvoraka u centru) nisam primetio neki veliku razliku o odnosu na originalnu sliku, ali kada u masku laplacian dodam i dijagonalne elemente deo ispod kacige mi postane izrazen, dakle nabori na celu prezvise izrazeni, kao i pore na licu.
### Sto se tice filtriranja pomocu prostorne maske izabrao sam mi koeficijent koji mnozi detalje postavim na 1 preko toga mi se cini da se ivice oko ociju sve vise i vise izostravaju kao da ga je neko nasminkao. To filtar koji sam odlcuio da ostavim finalni.

In [ ]:
img_rgb = img_as_float(imread('miner.jpg'))

#Prelazak u YUV sistem
img_yuv = color.rgb2yuv(img_rgb)

#Izvlacenje Ravni V koja ce biti izlozena modifikaciji
img_y = img_yuv[:,:,0]

#Definisanje matrice za filtriranje
lowpass_mask = np.ones(shape=(3,3))/9

#Filtriranje, i zatim i oduzimanje filtriranog od pocetne slike
img_y_blurred = ndimage.correlate(img_y, lowpass_mask)
img_y_details = img_y - img_y_blurred

#Isticanje detalja
img_y_sharp = img_y + 1*img_y_details

#Vracanje nazad u RGB foramt
img_yuv_out = np.zeros(shape(img_yuv))
img_yuv_out[:,:,1:3] = img_yuv[:,:,1:3]
img_yuv_out[:,:,0] = img_y_sharp

img_rgb_out = color.yuv2rgb(img_yuv_out)

#Klipovanje jer smo vrsili modifikacije nad parametrima koji su u YUV sistemu pa da budemo sigurni da je sve pravilno
img_rgb_out = np.clip(img_rgb_out, 0, 1)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16,8), dpi=60)
ax = axes.ravel()

ax[0].imshow(img_rgb, vmin=0, vmax=1); ax[0].set_axis_off(); ax[0].set_title('Ulazna slika', fontsize=16)
ax[1].imshow(img_rgb_out, vmin=0, vmax=1); ax[1].set_axis_off(); ax[1].set_title('Isticanje visokih učestanosti', fontsize=16)

#Cuvanje finalne slike
imsave('miner_sharp.jpg',img_rgb_out)

## 3 <span style="color:yellow">Implementacija <em>dosCLAHE(imgIn, numTiles, limit)</em> funkcije koja realizuje adaptivnu ekvalizaciju histograma uz ogranicenje histograma.</span>

## Izvestaj 3.1 -   Deo koji se odnosi na dosCLAHE funkciju
### Moja funkcija ne radi kako treba. Ja sam prvo krenuo da funkciju pravim u HSV sistemu. Naime procitao sam specifikaciju ugradjene funkcije da ona slike prevodi u HSV sistem a onda barata nad sivom sivom slikom. Takva implementacija je meni radila medjutim onda sam pomislio da je to greska i pokusao da svoj kod vratim da radi nad ravnima RGB. Rezultat je vidljiv dakle javaljaju se neke crne senke na presecima presecima kvadrata. Promena osvetljenja kupea je donekle i dobra.Kako je sad aminut do dvanaest nemam vremena da vratim nazad na kada je davala ispravne slike. Sto se tice razlike sa ugradjenom funkcijom sporija je , i ima crne sare.

In [ ]:
def dosCLAHE(img_in, num_tiles = [8, 8], limit = 0.01):
    """
    "Contrast Limited Adaptive Histogram Equalization"
    __Author__--Veljko Ilic, veljkoilic86@gmail.com
    
    Implementacija ispod zahteva parametar imgIn kao obavezan, numTiles i limit
    imaju svoje podrazumevane vrednosti.
    
    Dodatno funkcija je napisana tako da X i Y rezolucija slike ne moraju biti celobrojni
    umnosci broja oblasti, po vertikali i horizontali, na koje se slika deli. U suprotnom
    slika ce biti prosirena ponavljanjem odgovarajucih kolona i redova na krajevima slike.
    
    Za ulaznu sliku se smatra da je 2D ili 3D matrica u uint8 formatu.
    Inputs
    
      imgIn    - Ulazna slika koju treba obraditi{Obavezan argument} 
      numTiles - Niz od 2 elementa. Broj blokova po vertikali i horizontali.{Podrazumevano [8,8]}
      limit    - Normalizovani Clip Limit.{Podrazumevano 0.01}  
    Outputs
    
      imgOut - Obradjena slika istih dimenzija kao i ulazna
    
    Example
    clahe_image = dosCLAHE(img_in)
    clahe_image = dosCLAHE(img_in, num_tiles = [6, 3])
    clahe_image = dosCLAHE(img_in, num_tiles = [9, 5], limit = 0.8)
    """
    #Ispitavanje validnosti unetih podataka
    if(img_in.ndim != 2 and img_in.ndim != 3):
        print("Ulazna slika mora biti 2D ili 3D matrica!");
    if(num_tiles[0]<=0 or num_tiles[1]<=0):
        print("Broj pravougaonika na koje se deli slika mora biti vezi od nule")
    if(limit<=0):
        print("ClipLimit mora biti pozitivan broj.");
        
    #Izvlacenje sirine, visine i broja dimenzija
    height, width, ndim = img_in.shape;
    
    #Kreiranje izlazne slike koja je po formatu ista ulaznoj
    img_out = np.zeros(img_in.shape,dtype=uint8);
    
    #Ukoliko visina ili sirina nije celobrojan umnozak broj pravo. koje zelim po visin ili siri slika se prosiri
    if(height % num_tiles[0] or width % num_tiles[1]):
        
        if(height % num_tiles[0]):
            x = num_tiles[0] -  height%num_tiles[0];
            for i in range (0,x):
                img_in = np.vstack((img_in, img_in[np.arange(height-1,height),:][:, np.arange(0,width)]));
                height+=1;
        if(width % num_tiles[1]):
            y = num_tiles[1] -  width%num_tiles[1];
            for i in range (0,y):
                img_in = np.hstack((img_in, img_in[np.arange(0,height),:][:, np.arange(width-1,width)]));
                width+=1;
    min_val = 0 
    max_val = 255
    
    #Pravljenje LOOK UP tabele inteziteta piksela u sve tri ravni
    L = 256
    bin_sz = np.floor(1+(max_val-min_val)/float(L))
    LUT = np.floor((np.arange(min_val,max_val+1)-min_val)/float(bin_sz))
    bins = LUT[img_in]
    
    x_size = height//num_tiles[0];
    y_size = width//num_tiles[1];
    
    hist = np.zeros((num_tiles[0],num_tiles[1],L))
    MN = height//num_tiles[0] * width //num_tiles[1];
    
    #Petlja za pravljenje histograma koji je matrica num_tiles[0]xnum_tiles[0]x256
    for i in range(0,num_tiles[0]):
        for j in range(0,num_tiles[1]):
            bin_ = int32(bins[np.arange(i*x_size,(i+1)*x_size),:][:,np.arange(j*y_size,(j+1)*y_size)])
            
            for k in range(0,x_size):
                for h in range(0,y_size):
                    hist[i,j,bin_[k,h,0]]+=1;
                    hist[i,j,bin_[k,h,1]]+=1;
                    hist[i,j,bin_[k,h,2]]+=1;
                    
                    
            #Noramlizacija histograma deljenjem vrednosti binova sa povrsinom slike
            hist[i,j,:] = hist[i,j,:]/MN/3;
            
    #Klipovanje histograma i njegova ponovna normalizacija tako da PDF ostane  JEDAN!!! 
    
    for i in range(0, num_tiles[0]):
        for j in range(0, num_tiles[1]):
            sum_ = 0;
            for k in range(0, L):
                if(hist[i,j,k] > limit):
                    hist[i,j,k] = limit
                sum_ += hist[i,j,k];     
            hist[i,j,:] = hist[i,j,:]/sum_;
    
    #Ekvalizacija histograma
    T_r = np.zeros((num_tiles[0], num_tiles[1], L))
    
    for i in range(0, num_tiles[0]):
        for j in range(0, num_tiles[1]):
            sum_k = 0;
            for k in range(0,L):
                sum_k += hist[i,j,k];
                T_r[i,j,k] = np.round((L-1)*sum_k);
              
    #Interpolacija   
    M = height // num_tiles[0];
    N = width  // num_tiles[1];
    
    for x in range(0, img_out.shape[0]):
        for y in range(0,  img_out.shape[1]):
            # x koord. centra bloka 
            xc = int(M*np.floor(x/M) + (M-1)/2) 
            # y koord. centra bloka
            yc = int(N*np.floor(y/N) + (N-1)/2) 
            
            i0 = int(xc//M);
            i1 = int(xc//M);
            j0 = int(yc//N);
            j1 = int(yc//N);
            
            #Trazenje koordinata P,Q,R i S transformacionih funckija 
            if((x-xc < 0) and (y-yc < 0)):
                i0 -=1; j0 -=1;   
            elif((x-xc >= 0) and (y-yc < 0)): 
                i1 +=1; j0 -=1;
            elif((x-xc < 0) and (y-yc >= 0)):        
                i0 -=1; j1 +=1;
            elif((x-xc >= 0) and (y-yc >= 0)):
                i1 +=1; j1 +=1;
            if(i0<0):
                i0+=1
            if(i1>=num_tiles[0]):
                i1-=1
            if(j0<0):
                j0+=1
            if(j1>=num_tiles[1]):
                j1-=1
            #Kako je T_r matrica 8*8*256 ako nam je i ili j manje od nule ili vece od 7 dakle onda se radi o nekim piselim
            #koji su na ivicam slike i za koje smatram da je odgovaracjuca Transformac funkcija zbog nepostojanja susednog 
            #kvadrata nula
            P = np.zeros(256);  a = 0;
            Q = np.zeros(256);  b = 0;
            R = np.zeros(256);  c = 0;
            S = np.zeros(256);  d = 0;
            
            #Odredjivanje tezinskih faktora i koeficijenta transforamcione funkcije bloka iz susedstva
            
            i0_c = M*i0 + (M-1)/2
            i1_c = M*i1 + (M-1)/2   
            j0_c = N*j0 + (N-1)/2
            j1_c = N*j1 + (N-1)/2
            P = T_r[i0,j0,:];
            Q = T_r[i0,j1,:];
            R = T_r[i1,j1,:]; 
            S = T_r[i1,j0,:];
            a = abs(y - j0_c);
            b = abs(j1_c - y);
            c = abs(x - i0_c);
            d = abs(i1_c - x);           
            
            for i in range(0,3):
                I = img_in[x,y,i]
                sh1 = 0;
                sh2 = 0;
                
                #Linearna intrepolacija po horizontali
                if((a+b)!=0 and (c+d)!=0):
                    sh1 = (b*P[I] +a*Q[I])/(a+b)
                    sh2 = (b*S[I] +a*R[I])/(a+b)    
                    s = 0;
                    s = (d+sh1 + c*sh2)/(c+d)   
                    img_out[x,y,i] = uint8(s);
                else:
                    img_out[x,y,i] = uint8(T_r[x//M,y//N,I])
                
    return img_out;

In [ ]:
import time
t = time.time();
img = dosCLAHE(imread('train.jpg'),num_tiles=[8,8],limit=0.001)
x = time.time();
imsave('new_train.jpg',img)
print(x-t)
figure;
imshow((img).astype(uint8));